# Welcome to Feature Engineering! #
В этом курсе вы узнаете об одном из самых важных шагов на пути к созданию отличной модели машинного обучения: *разработке функций*. Вы узнаете, как:
- определять, какие функции являются наиболее важными, с помощью *взаимной информации*
- создавать новые функции в нескольких реальных проблемных областях.
- - кодировать категории высокой мощности с помощью *целевой кодировки*
- создавать объекты сегментации с помощью *кластеризации по k-среднему значению*
- - разбивать вариации набора данных на объекты с помощью *анализа основных компонентов*

Из практических упражнений можно составить полноценную **[записную книжку](http://www.kaggle.com/ryanholbrook/feature-engineering-for-house-prices)**, в которой будут использованы все эти методы для подачи заявки на участие в конкурсе **[Цены на жилье: начало работы](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)**. После прохождения этого курса у вас появится несколько идей, которые вы сможете использовать для дальнейшего повышения своей производительности.

# The Goal of Feature Engineering #

Цель разработки функциональных возможностей заключается в том, чтобы ваши данные лучше соответствовали поставленной задаче.

Рассмотрите такие показатели "кажущейся температуры", как индекс теплопроводности и холодность ветра. Эти величины позволяют измерить "воспринимаемую" человеком температуру на основе температуры воздуха, влажности и скорости ветра, которые мы можем измерить непосредственно. Вы могли бы рассматривать кажущуюся температуру как результат своего рода функциональной инженерии, попытки сделать наблюдаемые данные более релевантными тому, что нас на самом деле волнует: как на самом деле ощущается воздух на улице!

Вы можете разработать функционал, чтобы:
- улучшить прогнозные характеристики модели
- сократить потребности в вычислениях или данных
- улучшить интерпретируемость результатов

# Руководящий принцип разработки функциональных возможностей #

Чтобы функция была полезной, она должна иметь отношение к целевому объекту, которое может быть изучено вашей моделью. Например, линейные модели способны изучать только линейные отношения. Итак, при использовании линейной модели ваша цель состоит в том, чтобы преобразовать объекты таким образом, чтобы их связь с целью была линейной.

Ключевая идея здесь заключается в том, что преобразование, которое вы применяете к объекту, по сути, становится частью самой модели. Допустим, вы пытались предсказать `Price` квадратных земельных участков, исходя из `Length` одной из сторон. Подгонка линейной модели непосредственно к `Length` дает плохие результаты: зависимость не является линейной.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/5D1z24N.png" width=300, alt="A scatterplot of Length along the x-axis and Price along the y-axis, the points increasing in a curve, with a poorly-fitting line superimposed.">
<figcaption style="textalign: center; font-style: italic"><center>A linear model fits poorly with only Length as feature.
</center></figcaption>
</figure>

Однако, если мы возведем в квадрат объект `Length`, чтобы получить `Area`, мы создадим линейную зависимость. Добавление `Area` к набору объектов означает, что эта линейная модель теперь может соответствовать параболе. Другими словами, возведение объекта в квадрат позволяет линейной модели подогнать его под квадратные объекты.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BLRsYOK.png" width=600, alt="Left: Area now on the x-axis. The points increasing in a linear shape, with a well-fitting line superimposed. Right: Length on the x-axis now. The points increase in a curve as before, and a well-fitting curve is superimposed.">
<figcaption style="textalign: center; font-style: italic"><center><strong>Left:</strong> Они намного лучше вписываются в местность. <strong>Right:</strong> Что также улучшает подгонку по длине.
</center></figcaption>
</figure>

Это должно показать вам, почему время, затраченное на разработку функций, может приносить такую высокую отдачу. Какие бы взаимосвязи ваша модель не могла изучить, вы можете создать сами с помощью преобразований. Разрабатывая свой набор функций, подумайте о том, какую информацию можно было бы использовать для достижения наилучшей производительности вашей модели.

# Example - Concrete Formulations #

Чтобы проиллюстрировать эти идеи, мы увидим, как добавление нескольких синтетических признаков к набору данных может повысить эффективность прогнозирования модели случайного леса.

Набор данных [*Concrete*](https://www.kaggle.com/sinamhd9/concrete-comprehensive-strength) содержит различные составы бетона и *compressive strength* получаемого продукта, которая является показателем того, какую нагрузку может выдержать данный вид бетона. Задача этого набора данных - предсказать прочность бетона на сжатие с учетом его состава.

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

df = pd.read_csv("concrete.csv")
df.head()

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,CompressiveStrength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Здесь вы можете ознакомиться с различными ингредиентами, входящими в состав каждого вида бетона. Чуть позже мы увидим, как добавление некоторых дополнительных синтетических характеристик, полученных на их основе, может помочь модели изучить важные взаимосвязи между ними.

Сначала мы установим исходные условия, обучив модель на необработанном наборе данных. Это поможет нам определить, действительно ли полезны наши новые функции.

Установление таких базовых показателей является хорошей практикой в начале процесса разработки функций. Оценка базовых показателей может помочь вам решить, стоит ли сохранять ваши новые функции или вам следует отказаться от них и, возможно, попробовать что-то другое.

In [ ]:
X = df.copy()
y = X.pop("CompressiveStrength")

# Базовая модель обучения и оценки
baseline = RandomForestRegressor(criterion="absolute_error", random_state=0)
baseline_score = cross_val_score(
    baseline, X, y, cv=5, scoring="neg_mean_absolute_error"
)
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score: {baseline_score:.4}")

MAE Baseline Score: 8.272


Если вы когда-нибудь готовили дома, то, возможно, знаете, что соотношение (*ratio*) ингредиентов в рецепте обычно является лучшим показателем того, каким получится блюдо, чем их абсолютное количество. Мы можем предположить, что соотношение вышеперечисленных компонентов будет хорошим показателем `CompressiveStrength`.

Приведенная ниже ячейка добавляет в набор данных три новых параметра соотношения.

In [ ]:
X = df.copy()
y = X.pop("CompressiveStrength")

# Создание синтетических объектов
X["FCRatio"] = X["FineAggregate"] / X["CoarseAggregate"]
X["AggCmtRatio"] = (X["CoarseAggregate"] + X["FineAggregate"]) / X["Cement"]
X["WtrCmtRatio"] = X["Water"] / X["Cement"]

# Модель обучения и оценки на основе набора данных с дополнительными функциями соотношения
model = RandomForestRegressor(criterion="absolute_error", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

MAE Score with Ratio Features: 7.948


И, конечно же, производительность повысилась! Это свидетельствует о том, что новые функции ratio предоставляют модели важную информацию, которую она не обнаруживала раньше.

# Продолжение #

Мы убедились, что разработка новых функций может повысить производительность модели.  Но как вы определяете объекты в наборе данных, которые было бы полезно объединить?  [***Откройте для себя полезные функции***](https://www.kaggle.com/ryan holbrook/mutual-information) с помощью mutual information.